# Train With Crops

This notebook is an attempt to convert Chris's excellent notebook [here](https://www.kaggle.com/code/cdeotte/train-with-crops-lb-0-63) from Tensorflow/Keras to PyTorch. Some code cells that did not use Tensorflow/Keras were re-used with no edits made.

**Everything is based on Chris's notebook.** <br>
All credit belongs to the original author!

## Objective

- In this kernel we use a trick that we learned in Kaggle's Steel Competition. Since segmentation neural networks are all convolutions, you can train with one input size and predict with a different input size (explained in detail [here][1]). First we will resize all training and test images into size `700x1050` from their original `1400x2100`. Next we will train with random `352x512` crops and then feed full test images (`700x1050` images) into our network and get full segmentation masks!

[1]: https://www.kaggle.com/c/severstal-steel-defect-detection/discussion/114321

# Imports

In [ ]:
import os
import gc
import cv2
import timm
import time
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from PIL import Image 
import albumentations
from tqdm import tqdm
import matplotlib.pyplot as plt, time

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, accuracy_score

import segmentation_models_pytorch as smp
from segmentation_models_pytorch.losses import DiceLoss

from utils import mask2rleX, rle2maskX, mask2contour, clean, dice_coef6

# Config

In [ ]:
class cfg:
    lr = 1e-3
    n_epochs = 4
    batch_size = 8
    n_workers = 4
    n_folds = 3
    seed = 42
    n_classes = 4
    pred_batch_size = 2
    pred_width = 1024
    pred_height = 672
    crop_width = 512
    crop_height = 352
    shrink = 2
    orig_width = 2100
    orig_height = 1400
    train_img_folder = '../data/understanding_clouds/train_images'
    test_img_folder = '../data/understanding_clouds/test_images'
    net = 'resnet18'
    base_model_folder = './drive/MyDrive/understanding_clouds_models'
    train_crops_folder = './drive/MyDrive/understanding_clouds_models/train_with_crops'

# Load Data

In [ ]:
sub = pd.read_csv('../data/understanding_clouds/sample_submission.csv')
sub['Image'] = sub['Image_Label'].map(lambda x: x.split('.')[0])

PATH = '../data/understanding_clouds/train_images/'
train = pd.read_csv('../data/understanding_clouds/train.csv')
train['Image'] = train['Image_Label'].map(lambda x: x.split('.')[0])
train['Label'] = train['Image_Label'].map(lambda x: x.split('_')[1])
train2 = pd.DataFrame({'Image':train['Image'][::4]})
train2['e1'] = train['EncodedPixels'][::4].values
train2['e2'] = train['EncodedPixels'][1::4].values
train2['e3'] = train['EncodedPixels'][2::4].values
train2['e4'] = train['EncodedPixels'][3::4].values
train2.set_index('Image',inplace=True,drop=True)
train2.fillna('',inplace=True); train2.head()
train2[['d1','d2','d3','d4']] = (train2[['e1','e2','e3','e4']]!='').astype('int8')
train2.head()

In [ ]:
target_cols = train2.iloc[:, 4:8].columns.tolist()
target_cols

# Dataset

This data generator outputs random crops of size 352x512. These crops are taken from the original 1400x2100 training images after they are resized to 700x1050 pixels. The masks are cropped to match the image crops. Also we have horizontal and vertical flip augmentation.

In [ ]:
class CloudDataset(Dataset):
    def __init__(self, df, transform=None, mode='train', height=cfg.crop_height, width=cfg.crop_width, shrink=cfg.shrink):
        self.df = df
        self.transform = transform
        self.mode = mode
        if self.mode == 'train' or self.mode == 'display':
            self.labels = self.df[target_cols].values
        self.height = height
        self.width = width
        self.shrink = shrink
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        if self.mode == 'train': 
            msk = np.empty((self.height,self.width,4),dtype=np.int8)
            crp = np.zeros((2),dtype=np.int16)

            a = np.random.randint(0,cfg.orig_width//self.shrink-self.width+1)
            b = np.random.randint(0,cfg.orig_height//self.shrink-self.height+1)
            
            image_id = self.df.index[index]
            path = f'{cfg.train_img_folder}/{image_id}.jpg'

            if os.path.exists(path):
                img = cv2.imread(path, cv2.IMREAD_COLOR)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)              
                img = cv2.resize(img,(cfg.orig_width//self.shrink,cfg.orig_height//self.shrink),interpolation=cv2.INTER_AREA)
                img = img[b:self.height+b,a:self.width+a]

                for j in range(1, 5):
                    rle = self.df['e'+str(j)][index]
                    mask = rle2maskX(rle, shrink=self.shrink)
                    msk[:,:,j-1] = mask[b:self.height+b,a:self.width+a]

                if self.transform is not None:
                    res = self.transform(image=img, mask=msk)
                    img = res['image']
                    msk = res['mask']

                img = img.astype(np.float32)
                img = img.transpose(2,0,1)
                
                return torch.tensor(img), torch.tensor(msk)
        
        elif self.mode == 'display':
            msk = np.empty((self.height,self.width,4),dtype=np.int8)
            crp = np.zeros((2),dtype=np.int16)

            a = np.random.randint(0,cfg.orig_width//self.shrink-self.width+1)
            b = np.random.randint(0,cfg.orig_height//self.shrink-self.height+1)
            
            crp[0] = a; crp[1] = b
            
            image_id = self.df.index[index]
            path = f'{cfg.train_img_folder}/{image_id}.jpg'

            if os.path.exists(path):
                img = cv2.imread(path, cv2.IMREAD_COLOR)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)              
                img = cv2.resize(img,(cfg.orig_width//self.shrink,cfg.orig_height//self.shrink),interpolation=cv2.INTER_AREA)
                img = img[b:self.height+b,a:self.width+a]

                for j in range(1, 5):
                    rle = self.df['e'+str(j)][index]
                    mask = rle2maskX(rle, shrink=self.shrink)
                    msk[:,:,j-1] = mask[b:self.height+b,a:self.width+a]

                if self.transform is not None:
                    res = self.transform(image=img, mask=msk)
                    img = res['image']
                    msk = res['mask']

                img = img.astype(np.float32)
                img = img.transpose(2,0,1)

                
                return torch.tensor(img), torch.tensor(msk), crp
        
        elif self.mode == 'predict':
            a = (2100//self.shrink-self.width)//2
            b = (1400//self.shrink-self.height)//2
            
            image_id = self.df.index[index]
            path = f'{cfg.test_img_folder}/{image_id}.jpg'
            
            if os.path.exists(path):
                img = cv2.imread(path, cv2.IMREAD_COLOR)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)              
                img = cv2.resize(img,(cfg.orig_width//self.shrink,cfg.orig_height//self.shrink),interpolation=cv2.INTER_AREA)
                img = img[b:self.height+b,a:self.width+a]

                img = img.astype(np.float32)
                img = img.transpose(2,0,1)
                
                return torch.tensor(img)

# Augmentations

In [ ]:
transforms_train = albumentations.Compose([
  albumentations.HorizontalFlip(p=0.5),
  albumentations.VerticalFlip(p=0.5),
  albumentations.Normalize(),
])

transforms_valid = albumentations.Compose([
  albumentations.Normalize(),
])

Below we display examples. The image on the left is the original image. The yellow rectangle is an original mask. The black rectangle is a random crop. The image on the right is the random crop outputted by the data generator. Notice how the original mask is cropped too.

In [ ]:
train_ds = CloudDataset(train2[:8], mode='display', transform=transforms_train)
train_loader = DataLoader(train_ds, batch_size=1, shuffle=False)

In [ ]:
types = ['Fish','Flower','Gravel','Sugar']

for step, batch in enumerate(train_loader):
    plt.figure(figsize=(15,8))

    # RANDOMLY PICK CLOUD TYPE TO DISPLAY FROM NON-EMPTY MASKS
    idx = np.argwhere( train2.loc[train2.index[step],['d1','d2','d3','d4']].values==1 ).flatten()
    d = np.random.choice(idx)+1

    # DISPLAY ORIGINAL
    img = Image.open(PATH+train2.index[step]+'.jpg'); img=np.array(img)
    mask = rle2maskX( train2.loc[train2.index[step],'e'+str(d)] )
    contour = mask2contour( mask,10 )
    img[contour==1,:2]=255
    diff = np.ones((1400,2100,3),dtype=np.int)*255-img.astype(int)
    img=img.astype(int); img[mask==1,:] += diff[mask==1,:]//6
    mask = np.zeros((1400,2100))
    a = batch[2][0,1]*2
    b = batch[2][0,0]*2
    mask[a:a+2*352,b:b+2*512]=1
    mask = mask2contour(mask,20)
    img[mask==1,:]=0
    plt.subplot(1,2,1); 
    plt.title('Original - '+train2.index[step]+'.jpg - '+types[d-1])
    plt.imshow(img);

    # DISPLAY RANDOM CROP
    img = batch[0].cpu().detach().numpy()
    mask = batch[1].cpu().detach().numpy()
    img = img[0].transpose(1,2,0)
    mask = mask[0][:,:,d-1]
    contour = mask2contour( mask )
    img[contour==1,:2]=255
    diff = np.ones((352,512,3),dtype=np.int)*img
    img=img; img[mask==1,:] += diff[mask==1,:]//6
    plt.subplot(1,2,2)
    plt.title('Training Crop - '+train2.index[step]+'.jpg - '+types[d-1])
    plt.imshow( img ); 
    plt.show()

# Metric

In [ ]:
def dice_coef_metric(probabilities: torch.Tensor,
                     truth: torch.Tensor,
                     treshold: float = 0.5,
                     eps: float = 1e-9) -> np.ndarray:
    """
    Calculate Dice score for data batch.
    Params:
        probobilities: model outputs after activation function.
        truth: truth values.
        threshold: threshold for probabilities.
        eps: additive to refine the estimate.
        Returns: dice score aka f1.
    """
    scores = []
    num = probabilities.shape[0]
    predictions = (probabilities >= treshold).float()
    assert(predictions.shape == truth.shape)
    for i in range(num):
        prediction = predictions[i]
        truth_ = truth[i]
        intersection = 2.0 * (truth_ * prediction).sum()
        union = truth_.sum() + prediction.sum()
        if truth_.sum() == 0 and prediction.sum() == 0:
            scores.append(1.0)
        else:
            scores.append((intersection + eps) / union)
    return np.mean(scores)

# Loss Function

In [ ]:
def dice_loss(input, target):
    smooth = 1.

    iflat = input.reshape(-1)
    tflat = target.reshape(-1)
    intersection = (iflat * tflat).sum()
    
    return 1 - ((2. * intersection + smooth) /
              (iflat.sum() + tflat.sum() + smooth))

# Train and Validation Functions

In [ ]:
def train_func(model, loader, optimizer, loss_func, dice_coef_metric, device):
    model.train()

    losses = []
    dice = []

    bar = tqdm(enumerate(loader), total=len(loader))

    for step, (data, targets) in bar:
        data = data.to(device)
        targets = targets.permute(0, 3, 1, 2).to(device)

        optimizer.zero_grad()
        outputs = model(data)

        loss = loss_func(outputs, targets)
        
        loss.backward()
        optimizer.step()

        dice_scores = dice_coef_metric(outputs.detach().cpu(), targets.detach().cpu(), 0.5)
    
        losses.append(loss.item())
        dice.append(dice_scores)
        smooth_loss = np.mean(losses[-30:])
        bar.set_description(f'loss: {loss.item():.5f}, smth: {smooth_loss:.5f}, dice: {dice_scores:.5f}')

    return np.array(losses).mean(), np.array(dice).mean()


def valid_func(model, loader, loss_func, dice_coef_metric, device):
    model.eval()

    losses = []
    dice = []

    bar = tqdm(enumerate(loader), total=len(loader))

    with torch.no_grad():
        for step, (data, targets) in bar:
            data = data.to(device)
            targets = targets.permute(0, 3, 1, 2).to(device)

            outputs = model(img)
            loss = loss_func(outputs, msk)

            dice_scores = dice_coef_metric(outputs.detach().cpu(), targets.detach().cpu(), 0.5)

            losses.append(loss.item())
            dice.append(dice_scores)
            smooth_loss = np.mean(losses[-30:])
            bar.set_description(f'loss: {loss.item():.5f}, smth: {smooth_loss:.5f}, dice: {dice_scores:.5f}')

    return np.array(losses).mean(), np.array(dice).mean()

# Create K-Folds

In [ ]:
kf = KFold(cfg.n_folds, shuffle=True, random_state=cfg.seed)
train2["fold"] = -1
for i, (train_idx, valid_idx) in enumerate(kf.split(train2)):
    train2.loc[train2.index[valid_idx], "fold"] = i
train2.to_csv('folds.csv', index=False)
train2.head()

# Build Segmentation Model
We will build a segmentation model using Qubvel's Pytorch Segmentation models [here][1]. Our architecture will be FPN (feature pyramid network) and our backbone will be ResNet18. We will use Dice loss and Adam optimizer with learning rate 1e-3. Our metric will be Dice coef.

[1]: https://github.com/qubvel/segmentation_models.pytorch

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = smp.FPN(
    encoder_name=cfg.net,       
    encoder_weights="imagenet",     
    classes=cfg.n_classes, 
    activation='sigmoid'                     
).to(device)

optimizer = optim.Adam(model.parameters(), lr=cfg.lr)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, cfg.n_epochs)
loss_fn = dice_loss

# Train Segmentation Model on Crops
We will train with `352x512` crops, `batch_size=8`, and use 3-Fold validation. During training we predict OOF. We predict `test.csv` afterwards by saving the 3 models from the 3 folds. We will train each fold for 4 epochs. In our predicted oof masks, all pixel probabilities over 0.4 will be converted to 1 and less than 0.4 to 0. Any mask with fewer than `4*20000` pixels (predicted on `700x1050` image) will be regarded as no mask.

In [ ]:
oof = np.empty_like(train2[['e1','e2','e3','e4']].values)

for fold in range(cfg.n_folds):
    train_idx = np.where((train2["fold"] != fold))[0]
    valid_idx = np.where((train2["fold"] == fold))[0]

    print(); print('#'*10,'FOLD',fold,'#'*10)
    print('Train on',len(train_idx),'Validate on',len(valid_idx))

    train_ds = CloudDataset(train2.loc[train2.index[train_idx]], transform=transforms_train)
    valid_ds = CloudDataset(train2.loc[train2.index[valid_idx]], transform=transforms_valid)
    
    train_loader = DataLoader(train_ds, batch_size=cfg.batch_size, num_workers=cfg.n_workers, shuffle=True)
    valid_loader = DataLoader(valid_ds, batch_size=cfg.batch_size, num_workers=cfg.n_workers, shuffle=False)

    best_dice = 0.0

    for epoch in range(1, cfg.n_epochs+1):
        loss_train, dice_train = train_func(model, train_loader, optimizer, loss_fn, dice_coef_metric, device)
        scheduler.step(epoch)
        loss_valid, dice_valid = valid_func(model, valid_loader, loss_fn, dice_coef_metric, device)

        content = time.ctime() + ' ' + f'Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, loss_train: {loss_train:.5f}, loss_valid: {loss_valid:.5f}, dice: {dice_valid:.6f}.'
        print(content)

        if dice_valid > best_dice:
            print(f'accuracy ({best_dice:.6f} --> {dice_valid:.6f}). Saving model ...')
            torch.save(model.state_dict(), f'{cfg.train_crops_folder}/{cfg.net}_best_dice_fold{fold}.pth')
            best_dice = dice_valid   
  
    # PREDICT OOF
    print('Predict OOF: ',end='')
    valid_ds = CloudDataset(train2.loc[train2.index[valid_idx]], width=cfg.pred_width, height=cfg.pred_height, mode='predict', transform=transforms_valid)
    valid_loader = DataLoader(valid_ds, batch_size=cfg.pred_batch_size, num_workers=cfg.n_workers, shuffle=False)
    model.eval()

    bar = tqdm(enumerate(valid_loader), total=len(valid_loader))

    with torch.no_grad():
        for step, (data, targets) in bar:
            data = data.to(device)
            targets = targets.permute(0, 3, 1, 2).to(device)

            outputs = model(data)
            out = outputs.permute(0, 2, 3, 1)
            for j in range(out.shape[0]):
                for i in range(out.shape[-1]):
                    mask = out[j,:,:,i]
                    mask = mask.cpu().detach().numpy()
                    mask = mask > 0.4
                    rle =''
                    if np.sum(mask)>4*20000:
                        rle = mask2rleX(mask)
                    oof[valid_idx[2*step+j],i] = rle

# Save OOF

In [ ]:
np.save(f'{cfg.base_model_folder}/oof_seg_preds_3fold.npy', oof)

# Evaluate Segmentation Model using OOF
We will evaluate OOF using Kaggle's Dice metric. For post processing, we will remove any contiguous piece of predicted mask with fewer than 20000 pixels (predicted on `350x525` image). And we will remove any mask with less than 0.5 probability as determined by our cloud classifer from our previous notebook [here][1]

[1]: https://www.kaggle.com/cdeotte/cloud-bounding-boxes-cv-0-58

In [ ]:
# LOAD CLASSIFICATION PREDICTIONS FROM PREVIOUS KERNEL
# https://www.kaggle.com/cdeotte/cloud-bounding-boxes-cv-0-58
for k in range(1,5): train2['o'+str(k)] = 0
train2[['o1','o2','o3','o4']] = np.load(f'{cfg.base_model_folder}/oof.npy')[:len(train2),]

# LOAD OOF SEGMENTATION PREDICTIONS FROM 3-FOLD ABOVE
for k in range(1,5): train2['ee'+str(k)] = ''
train2[['ee1','ee2','ee3','ee4']] = oof
for k in range(1,5): train2['ee'+str(k)] = train2['ee'+str(k)].map(clean)

# COMPUTE KAGGLE DICE
th = [0.5,0.5,0.5,0.5]
for k in range(1,5):
    train2['ss'+str(k)] = train2.apply(lambda x:dice_coef6(x['e'+str(k)],x['o'+str(k)],x['ee'+str(k)],th[k-1]),axis=1)
    dice = np.round( train2['ss'+str(k)].mean(),3 )
    print(types[k-1],': Kaggle Dice =',dice)
dice = np.round( np.mean( train2[['ss1','ss2','ss3','ss4']].values ),3 )
print('Overall : Kaggle Dice =',dice)

# View OOF Examples
Below yellow outlines are true masks and blue outlines (with shaded insides) are predicted masks. The Dice score for each predicted mask is displayed above each image.

In [ ]:
for d in range(1,5):
    print('#'*27); print('#'*5,types[d-1].upper(),'CLOUDS','#'*7); print('#'*27)
    plt.figure(figsize=(20,15)); k=0
    for kk in range(9):
        plt.subplot(3,3,kk+1)
        while (train2.loc[train2.index[k],'e'+str(d)]==''): k += 1
        f = train2.index[k]+'.jpg'
        img = Image.open(PATH+f); img = img.resize((525,350)); img = np.array(img)
        rle1 = train2.loc[train2.index[k],'e'+str(d)]; mask = rle2maskX(rle1,shrink=4)
        contour = mask2contour(mask,5); img[contour==1,:2] = 255
        rle2 = train2.loc[train2.index[k],'ee'+str(d)]; mask = rle2maskX(rle2,shape=(525,350))
        contour = mask2contour(mask,5); img[contour==1,2] = 255
        diff = np.ones((350,525,3),dtype=np.int)*255-img
        img=img.astype(int); img[mask==1,:] += diff[mask==1,:]//4
        dice = np.round( dice_coef6(rle1,1,rle2,0),3 )
        plt.title(f+'  Dice = '+str(dice)+'   Yellow true, Blue predicted')
        plt.imshow(img); k += 1
    plt.show()

# Predict Test Images
We load the models from our 3 folds above and use them to predict `test.csv`. Note that we must choose an input size that is divisble by 32. Therefore we choose `672x1024` instead of `700x1050`. We segment the middle of each test image ignoring the 14 pixel wide border around the edge.

In [ ]:
sub2 = pd.DataFrame({'Image':sub['Image'][::4]}).reset_index(drop=True)
sub2.set_index('Image',inplace=True,drop=True)
sub2.fillna('',inplace=True); 
sub2.head()

In [ ]:
print('Computing masks for',len(sub)//4,'test images with 3 models'); sub.EncodedPixels = ''
test_ds = TestCloudDataset(sub2, width=cfg.pred_width, height=cfg.pred_height, mode='predict', transform=transforms_valid)
test_loader = DataLoader(test_ds, batch_size=cfg.pred_batch_size, num_workers=cfg.n_workers, shuffle=False)

model = smp.FPN(
    encoder_name="resnet18",       
    encoder_weights="imagenet",     
    classes=cfg.n_classes, 
    activation='sigmoid'                     
).to(device)

m1 = model
m1.load_state_dict(torch.load(f'{cfg.train_crops_folder}/{cfg.net}_best_dice_fold0.pth'))
m1 = m1.to(device)

m2 = model
m2.load_state_dict(torch.load(f'{cfg.train_crops_folder}/{cfg.net}_best_dice_fold1.pth'))
m2 = m2.to(device)

m3 = model
m3.load_state_dict(torch.load(f'{cfg.train_crops_folder}/{cfg.net}_best_dice_fold2.pth'))
m3 = m3.to(device)

m1.eval()
m2.eval()
m3.eval()

bar = tqdm(enumerate(test_loader), total=len(test_loader))
with torch.no_grad():
    for step, data in bar:
        data = data.to(device)

        outputs = m1(data)
        outputs += m2(data)
        outputs += m3(data)

        outputs /= 3.0
        out = outputs.permute(0, 2, 3, 1)
        for j in range(out.shape[0]):
            for i in range(out.shape[-1]):
                mask = out[j,:,:,i]
                mask = mask.cpu().detach().numpy()
                mask = mask > 0.4
                rle = ''
                if np.sum(mask)>4*20000:
                    rle = mask2rleX(mask)
                sub.iloc[4*(2*step+j)+i, 1] = rle

In [ ]:
# LOAD CLASSIFICATION PREDICTIONS FROM PREVIOUS KERNEL
# https://www.kaggle.com/cdeotte/cloud-bounding-boxes-cv-0-58
sub['p'] = np.load(f'{cfg.base_model_folder}/preds.npy').reshape((-1))[:len(sub)]
sub.loc[sub.p<0.5,'EncodedPixels'] = ''

sub.EncodedPixels = sub.EncodedPixels.map(clean)
sub[['Image_Label','EncodedPixels']].to_csv('submission.csv',index=False)
sub.head(25)